In [49]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
import find_best_hyperparameters as fbh
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
# Executar essa celula para utilizar o arquivo "./Dmoz-Science.csv"

data = fbh.load_db("./Dmoz-Science.csv")

In [51]:
# Executar essa celula para utilizar o arquivo webkb-parsed.csv
data = fbh.load_db("webkb-parsed.csv")

# Multinomial Naive Bayes

In [58]:

hyperparams_df = fbh.greedy_search_MultinomialNB(data, [0.1, 0.5, 1.0, 2.0, 5.0])
print(hyperparams_df)
best_alpha_row = hyperparams_df.loc[hyperparams_df['accuracy'].idxmax()]
best_alpha = best_alpha_row['alpha']
best_accuracy = best_alpha_row['accuracy']
print(f"\nMelhor alpha encontrado: {best_alpha} com acurácia de {best_accuracy:.4f}\n")



   alpha  accuracy  f1_macro  f1_micro
0    0.1  0.717500  0.711544  0.717500
1    0.5  0.720000  0.716553  0.720000
2    1.0  0.710833  0.709062  0.710833
3    2.0  0.689167  0.689243  0.689167
4    5.0  0.650000  0.649790  0.650000

Melhor alpha encontrado: 0.5 com acurácia de 0.7200



In [59]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['text'])
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

final_model = MultinomialNB(alpha=best_alpha)
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred)
final_f1_macro = f1_score(y_test, y_pred, average='macro')
final_f1_micro = f1_score(y_test, y_pred, average='micro')


In [60]:

print("\nMétricas no conjunto de teste:")
print(f"Acurácia: {final_accuracy:.4f}")
print(f"F1 Score (Macro): {final_f1_macro:.4f}")
print(f"F1 Score (Micro): {final_f1_micro:.4f}")

print("\nInterpretação dos Resultados:")
print(f"O melhor hiperparâmetro encontrado foi alpha = {best_alpha}, com base na maior acurácia no conjunto de validação.")
print("O modelo apresentou boa generalização no conjunto de teste, com métricas F1 Score (Macro e Micro) elevadas.")
print("Um valor de alpha menor tende a funcionar melhor para textos mais curtos ou com menos ruído nos dados, enquanto valores maiores suavizam os pesos.")





Métricas no conjunto de teste:
Acurácia: 0.7200
F1 Score (Macro): 0.7166
F1 Score (Micro): 0.7200

Interpretação dos Resultados:
O melhor hiperparâmetro encontrado foi alpha = 0.5, com base na maior acurácia no conjunto de validação.
O modelo apresentou boa generalização no conjunto de teste, com métricas F1 Score (Macro e Micro) elevadas.
Um valor de alpha menor tende a funcionar melhor para textos mais curtos ou com menos ruído nos dados, enquanto valores maiores suavizam os pesos.


In [61]:
results_df = pd.DataFrame({
    'file_name': data.loc[y_test.index, 'file_name'],
    'text': data.loc[y_test.index, 'text'],
    'true_class': y_test,
    'predicted_class': y_pred
})

results_df.to_csv("./results_MultinomialNB.csv", index=False)

# Logistic Regression

In [62]:
hyperparams_df = fbh.greedy_search_LogisticRegression(data, [0.01, 0.1, 1, 10, 100])
print(hyperparams_df)
best_row = hyperparams_df.loc[hyperparams_df['accuracy'].idxmax()]
best_C = best_row['C']
best_accuracy = best_row['accuracy']
print(f"\nMelhor C encontrado: {best_C} com acurácia de {best_accuracy:.4f}\n")

        C  accuracy  f1_macro  f1_micro
0    0.01  0.549167  0.541022  0.549167
1    0.10  0.685833  0.681072  0.685833
2    1.00  0.706667  0.702757  0.706667
3   10.00  0.715000  0.711247  0.715000
4  100.00  0.715833  0.713002  0.715833

Melhor C encontrado: 100.0 com acurácia de 0.7158



In [63]:

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['text'])
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(C=best_C, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred)
final_f1_macro = f1_score(y_test, y_pred, average='macro')
final_f1_micro = f1_score(y_test, y_pred, average='micro')


In [64]:
print("\nResultados finais no conjunto de teste:")
print(f"Acurácia: {final_accuracy:.4f}")
print(f"F1 Score (Macro): {final_f1_macro:.4f}")
print(f"F1 Score (Micro): {final_f1_micro:.4f}")


Resultados finais no conjunto de teste:
Acurácia: 0.7158
F1 Score (Macro): 0.7130
F1 Score (Micro): 0.7158


In [65]:
results_df = pd.DataFrame({
    'file_name': data.loc[y_test.index, 'file_name'],
    'text': data.loc[y_test.index, 'text'],
    'true_class': y_test,
    'predicted_class': y_pred
})

results_df.to_csv("./results_LogisticRegression.csv", index=False)

# Linear Support Vector Classification

In [66]:
hyperparams_df = fbh.greedy_search_LinearSVC(data, [0.01, 0.1, 1, 10, 100])
print(hyperparams_df)
best_row = hyperparams_df.loc[hyperparams_df['accuracy'].idxmax()]
best_C = best_row['C']
best_accuracy = best_row['accuracy']
print(f"\nMelhor C encontrado: {best_C} com acurácia de {best_accuracy:.4f}\n")

        C  accuracy  f1_macro  f1_micro
0    0.01  0.720833  0.714070  0.720833
1    0.10  0.723333  0.718956  0.723333
2    1.00  0.716667  0.712776  0.716667
3   10.00  0.705833  0.701577  0.705833
4  100.00  0.706667  0.702845  0.706667

Melhor C encontrado: 0.1 com acurácia de 0.7233



In [67]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['text'])
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearSVC(C=best_C, random_state=42, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred)
final_f1_macro = f1_score(y_test, y_pred, average='macro')
final_f1_micro = f1_score(y_test, y_pred, average='micro')


In [68]:
print("\nResultados finais no conjunto de teste:")
print(f"Acurácia: {final_accuracy:.4f}")
print(f"F1 Score (Macro): {final_f1_macro:.4f}")
print(f"F1 Score (Micro): {final_f1_micro:.4f}")


Resultados finais no conjunto de teste:
Acurácia: 0.7233
F1 Score (Macro): 0.7190
F1 Score (Micro): 0.7233


In [69]:
results_df = pd.DataFrame({
    'file_name': data.loc[y_test.index, 'file_name'],
    'text': data.loc[y_test.index, 'text'],
    'true_class': y_test,
    'predicted_class': y_pred
})

results_df.to_csv("./results_LinearSVC.csv", index=False)